<a href="https://colab.research.google.com/github/Sejong-Kaggle-Challengers/Hyundong/blob/main/%5B%ED%9A%8C%EA%B8%B0%5D%ED%87%B4%EA%B7%BC%EC%8B%9C%EA%B0%84_%EB%B2%84%EC%8A%A4%EC%8A%B9%EC%B0%A8%EC%9D%B8%EC%9B%90_%EC%98%88%EC%B8%A1_%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd

In [3]:
train=pd.read_csv("/content/drive/MyDrive/DACON/01.26 퇴근시간 버스승차인원 예측 경진대회/data/train.csv")
test=pd.read_csv("/content/drive/MyDrive/DACON/01.26 퇴근시간 버스승차인원 예측 경진대회/data/test.csv")


In [9]:
print(train.shape)
train.head()

(415423, 21)


,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride
0,0,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,0.0,1.0,2.0,5.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2019-09-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,4.0,4.0,2.0,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
2,2,2019-09-01,4270000,시외,432,정존마을,33.48181,126.47352,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,3,2019-09-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.49252,0.0,17.0,6.0,26.0,14.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0
4,4,2019-09-01,4270000,시내,1646,중문관광단지입구,33.25579,126.41260,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [10]:
print(test.shape)
test.head()

(228170, 20)


,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff
0,415423,2019-10-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,4.0,4.0,7.0,2.0,9.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,415424,2019-10-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,6.0,6.0,1.0,8.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0
2,415425,2019-10-01,4270000,시외,432,정존마을,33.48181,126.47352,2.0,4.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,415426,2019-10-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.49252,1.0,11.0,18.0,8.0,26.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0
4,415427,2019-10-01,4270000,시내,1636,롯데호텔,33.24872,126.41032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415423 entries, 0 to 415422
Data columns (total 21 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id             415423 non-null  int64  
 1   date           415423 non-null  object 
 2   bus_route_id   415423 non-null  int64  
 3   in_out         415423 non-null  object 
 4   station_code   415423 non-null  int64  
 5   station_name   415423 non-null  object 
 6   latitude       415423 non-null  float64
 7   longitude      415423 non-null  float64
 8   6~7_ride       415423 non-null  float64
 9   7~8_ride       415423 non-null  float64
 10  8~9_ride       415423 non-null  float64
 11  9~10_ride      415423 non-null  float64
 12  10~11_ride     415423 non-null  float64
 13  11~12_ride     415423 non-null  float64
 14  6~7_takeoff    415423 non-null  float64
 15  7~8_takeoff    415423 non-null  float64
 16  8~9_takeoff    415423 non-null  float64
 17  9~10_takeoff   415423 non-nul

In [14]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228170 entries, 0 to 228169
Data columns (total 20 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id             228170 non-null  int64  
 1   date           228170 non-null  object 
 2   bus_route_id   228170 non-null  int64  
 3   in_out         228170 non-null  object 
 4   station_code   228170 non-null  int64  
 5   station_name   228170 non-null  object 
 6   latitude       228170 non-null  float64
 7   longitude      228170 non-null  float64
 8   6~7_ride       228170 non-null  float64
 9   7~8_ride       228170 non-null  float64
 10  8~9_ride       228170 non-null  float64
 11  9~10_ride      228170 non-null  float64
 12  10~11_ride     228170 non-null  float64
 13  11~12_ride     228170 non-null  float64
 14  6~7_takeoff    228170 non-null  float64
 15  7~8_takeoff    228170 non-null  float64
 16  8~9_takeoff    228170 non-null  float64
 17  9~10_takeoff   228170 non-nul

- id : 해당 데이터에서의 고유한 ID(train, test와의 중복은 없음)
- date : 날짜
- bus_route_id : 노선ID
- in_out : 시내버스, 시외버스 구분
- station_code : 해당 승하차 정류소의 ID
- station_name : 해당 승하차 정류소의 이름
- latitude : 해당 버스 정류장의 위도 (같은 정류장 이름이어도 버스의 진행 방향에 따라 다를 수 있음)
- longitude : 해당 버스 정류장의 경도 (같은 정류장 이름이어도 버스의 진행 방향에 따라 다를 수 있음)
- X~Y_ride : X:00:00부터 X:59:59까지 승차한 인원 수
- X~Y_takeoff : X:00:00부터 X:59:59까지 하차한 인원 수
- 18~20_ride : 18:00:00부터 19:59:59까지 승차한 인원 수 (train data에만 존재)

date, in_out을 학습 데이터로 변환

### pd.DataFrame.describe()
- 숫자형 (int, float) column들의 기술 통계량을 보여주는 함수

- 기술통계량이란?
 - 해당 column을 대표할 수 있는 통계값들을 의미
 
 
- 기술통계량 종류
 - count: 해당 column에서 비어 있지 않은 값의 개수
 - mean: 평균
 - std: 표준편차
 - min: 최솟값 (이상치 포함)
 - 25% (Q1): 전체 데이터를 순서대로 정렬했을 때, 아래에서 부터 1/4번째 지점에 있는 값
 - 50% (Q2): 중앙값 (전체 데이터를 순서대로 정렬했을 때, 아래에서 부터 2/4번째 지점에 있는 값)
 - 75% (Q3): 전체 데이터를 순서대로 정렬했을 때, 아래에서 부터 3/4번째 지점에 있는 값
 - max: 최댓값 (이상치 포함) 
 
 
 
- 이상치: 울타리 밖에 있는 부분을 이상치라고 정의함
   - 아래쪽 울타리: $Q_1$ - $1.5 * IQR$
   - 위쪽 울타리: $Q_3$ + $1.5 * IQR$
   - $IQR$ = $Q_3 - Q_1$
 
 
<img src="https://miro.medium.com/max/10125/1*NRlqiZGQdsIyAu0KzP7LaQ.png" width="700" height="500">

In [27]:
train.describe()

,id,bus_route_id,in_out,station_code,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride,date_weekday
count,415423.000000,4.154230e+05,415423.000000,4.154230e+05,415423.000000,415423.000000,415423.000000,415423.000000,415423.000000,415423.000000,415423.000000,415423.000000,415423.000000,415423.000000,415423.000000,415423.000000,415423.000000,415423.000000,415423.000000,415423.000000
mean,207711.000000,2.691935e+07,0.016665,4.209311e+04,33.420375,126.533574,0.305893,0.829699,0.815350,0.642475,0.599618,0.579393,0.112870,0.344870,0.516481,0.430922,0.408001,0.402874,1.242095,2.719349
std,119922.434776,3.924652e+06,0.128013,4.971504e+05,0.107996,0.140986,1.109766,2.255116,2.317561,1.959844,1.885941,1.942137,0.597714,1.279179,1.658850,1.485124,1.412839,1.446080,4.722287,2.017205
min,0.000000,4.270000e+06,0.000000,1.000000e+00,33.208350,126.165040,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,103855.500000,2.346000e+07,0.000000,3.200000e+02,33.291080,126.475780,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,207711.000000,2.803000e+07,0.000000,1.130000e+03,33.478850,126.529770,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
75%,311566.500000,3.003000e+07,0.000000,2.226000e+03,33.500020,126.575890,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,4.000000
max,415422.000000,3.282000e+07,1.000000,6.115101e+06,33.963640,126.965670,85.000000,94.000000,136.000000,78.000000,124.000000,99.000000,45.000000,66.000000,59.000000,65.000000,52.000000,81.000000,272.000000,6.000000


In [16]:
len(train['bus_route_id'].unique())


613

In [19]:
train[['bus_route_id', 'station_code']]

,bus_route_id,station_code
0,4270000,344
1,4270000,357
2,4270000,432
3,4270000,1579
4,4270000,1646
...,...,...
415418,32820000,1129
415419,32820000,1564
415420,32820000,2322
415421,32820000,3291


# 데이터 전처리

In [31]:
# Object -> datetime64

train['date'] = pd.to_datetime(train['date'], errors='raise')
test['date'] = pd.to_datetime(test['date'], errors='raise')
train['date_weekday'] = train['date'].dt.weekday
test['date_weekday'] = test['date'].dt.weekday

# 0: 월 1: 화 ,,, 6: 일

In [22]:
train['date'].head()

0   2019-09-01
1   2019-09-01
2   2019-09-01
3   2019-09-01
4   2019-09-01
Name: date, dtype: datetime64[ns]

In [23]:
train['date_weekday'].head()

0    6
1    6
2    6
3    6
4    6
Name: date_weekday, dtype: int64

In [24]:

#in_out 숫자로 변경 (시외 : 1, 시내 : 0)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['in_out'] = le.fit_transform(train['in_out'])
test['in_out'] = le.transform(test['in_out'])
train['in_out']

0         1
1         1
2         1
3         0
4         0
         ..
415418    0
415419    0
415420    0
415421    0
415422    0
Name: in_out, Length: 415423, dtype: int64

# 학습

In [25]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import lightgbm as lgb


In [28]:
features = ['bus_route_id',
'in_out',
'station_code',
'date_weekday',
'6~7_ride',
'7~8_ride',
'8~9_ride',
'9~10_ride',
'10~11_ride',
'11~12_ride']

target = ['18~20_ride']
features

['bus_route_id',
 'in_out',
 'station_code',
 'date_weekday',
 '6~7_ride',
 '7~8_ride',
 '8~9_ride',
 '9~10_ride',
 '10~11_ride',
 '11~12_ride']

In [33]:
X_train, X_test, Y_train = train[features], test[features], train[target]

In [26]:
model_dict = {
    'linear': LinearRegression(),
    'rf': RandomForestRegressor(random_state=0, n_jobs=-1),
    'lgbm': lgb.LGBMRegressor(random_state=0)
}

In [35]:
model_result = {}

for key in model_dict.keys():
  model_dict[key].fit(X_train, Y_train)

  model_result[key] = model_dict[key].predict(X_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


In [36]:
submission = pd.read_csv('/content/drive/MyDrive/DACON/01.26 퇴근시간 버스승차인원 예측 경진대회/data/submission_sample.csv')


In [37]:
lr_submit = submission.copy()
rf_submit = submission.copy()
lgbm_submit = submission.copy()

In [38]:
lr_submit['18~20_ride'] = model_result['linear']
rf_submit['18~20_ride'] = model_result['rf']
lgbm_submit['18~20_ride'] = model_result['lgbm']

In [40]:
lr_submit.to_csv('lr_submit.csv', index = False)
rf_submit.to_csv('rf_submit.csv', index = False)
lgbm_submit.to_csv('lgbm_submit.csv', index = False)